# Getting Started with Kubeflow-Fairing


Fairing is a python library that allows for deployment and management of kubernetes resources from scripts or notebooks

While fairing can be run from inside a notebook deployed on Kubeflow, it can also be run locally - allowing data scientists to use the machine, tools, and IDEs they prefer for developing their models while taking advantage of K8s for compute and deployment

_tl;dr - Fairing lets us build locally and deploy to K8s_

In [1]:
try:
    from kubeflow import fairing
except:
    !pip install kubeflow-fairing
    from kubeflow import fairing

## Core concepts:

* Fairing doesn't deploy resources directly to K8s

Fairing pushes containers to a container registry, then creates a deployment in K8s which pulls the configured container

This means we need a registry to work with. 

Working with registries / authentication is likely one of the more complex parts of getting Kubeflow and Fairing set up. Fairing has utilities to work and auth with varioius public clouds to make things a little easier.

For example:


```python
#AWS
fairing.cloud.aws.create_ecr_registry() #create an Elastic Container Registry
fairing.cloud.aws.add_aws_credentials() #get credentials

#Azure
fairing.cloud.azure.add_acr_config() #configure to Azure Container Registry
fairing.cloud.azure.AzureFileUploader() #move a file to AzureBlobStorage

#GCP
fairing.cloud.gcp.guess_project_name() #grab GCP Project Name
fairing.cloud.gcp.add_gcp_credentials() #add gcp service account credentials to path
```

We'll use a local registry as much as we can.

In [ ]:
DOCKER_REGISTRY = 'localhost:5000'

## 3 Parts of Fairing

1. Code

Models, processing code, etc

2. Builder

Defines how fairing is going to build an image to package your code

3. Deployer

Defines how fairing is going to deploy your code 


### 1. Preprocess Your Code

Build your code as your normally would.

Fairing has several ways to preprocess your code so it can interact with your code on K8s. We'll take a look at two:
* Define a function and run it on K8s
* Define a class and run a Training Job or Prediction Job



### 2. Build your container image

Fairing has lots of ways to package up your code. 

It can take a Python object and send that up to K8s for training or prediction

It can scrape through your notebook, only taking the cells you designate, then build a docker image based on your python environment

It can take an existing container and append your code and few utilities

### 3. Deploy your code

Fairing lets you access many of the training options that are natively deployed in kubeflow; such as tensorflow training, job based deployers, model serving with seldon or k8s

You can even deploy models to google's managed model services

#### Hello World on K8s with Fairing

In [2]:
# Our Basic Function
def train():
    print("hello world!")
    
train()

hello world!


We'll use an **Append Builder** in this case.

"Append" lets us piggyback on an existing image, just adding our code and the utilities to serve it - meaning lightning fast build times even with huge images.

As we only have a basic python function, we can grab a lightweight python image

In [2]:
base_image = 'registry.hub.docker.com/library/python:3.6-slim'

# Push controls whether the image is actually moved to the container registry
# If you would like to test your building process, you can change push to "False"

fairing.config.set_builder('append',base_image=base_image, registry=DOCKER_REGISTRY, push=True)


Next up is the **Deployer**

we'll use a 'job' deployer. Other options include:
- tfjob
- kfserving (works with kubeflow 0.7.0)
- gcp (sends job to google ml services)

In [ ]:
fairing.config.set_deployer('job')

In [6]:
remote_train = fairing.config.fn(train)
remote_train()

Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f10641854e0>
Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f10900d2048>
Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f10900d2048>
Building image using Append builder...
Creating docker context: /tmp/fairing_context_13x9cuar
/home/john/anaconda3/envs/kubeflow/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'registry.hub.docker.com/library/python:3.6-slim'
Image successfully built in 0.8022275949988398s.
Pushing image localhost:5000/fairing-job:E1CCC976...
Loading Docker credentials for repository 'localhost:5000/fairing-job:E1CCC976'
Uploading localhost:5000/fairing-job:E1CCC976
Layer sha256:009f8c0603342c9065b3f0bc297c67d477260fe4b06e8cf6c09c7dc3098f298a exists, skipping
Layer sha256:8d691f585fa8cec0eba196be460cfaffd69939

hello world!


Let's break that down:

```
Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f10641854e0>
Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f10900d2048>
Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f10900d2048>
```
This echos our setup process. A function preprocessor, with an append builder

```
Building image using Append builder...
Creating docker context: /tmp/fairing_context_13x9cuar
/home/john/anaconda3/envs/kubeflow/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'registry.hub.docker.com/library/python:3.6-slim'
Image successfully built in 0.8022275949988398s.
```
The append builder put together our image in just 0.8 seconds!

```
Pushing image localhost:5000/fairing-job:E1CCC976...
Loading Docker credentials for repository 'localhost:5000/fairing-job:E1CCC976'
Uploading localhost:5000/fairing-job:E1CCC976
```
Fairing tagged our container and sent it to our Registry, including a uniquie run ID so we can distinguish between models / jobs later on.

```
Layer sha256:009f8c0603342c9065b3f0bc297c67d477260fe4b06e8cf6c09c7dc3098f298a exists, skipping
Layer sha256:8d691f585fa8cec0eba196be460cfaffd69939782d6162986c3e0c5225d54f02 exists, skipping
Layer sha256:73c12d19d4644e83661f961761525e701b8f2f6f84405998e630367ee079916f exists, skipping
Layer sha256:49bdb6f85638342d5b9037282ef68bb619f2a3759b53fd2dc6de305b3212302d exists, skipping
Layer sha256:18a21a4ca4a00c5298021a29819ac268135b9a8d2ad987c8a24857282ebc3f07 exists, skipping
Layer sha256:9964ba7bfd8f3c355fe9e211f50135ca1f88745128774f3e6950aae0ba3f15b8 exists, skipping
Layer sha256:ba0564f7c814e62afaed4fce713c39a5c3d1b0b5d9c65298635ffe014296ebd0 pushed.
Finished upload of: localhost:5000/fairing-job:E1CCC976
Pushed image localhost:5000/fairing-job:E1CCC976 in 0.2118565339987981s.
```
Built the layers, and pushed into the registry

```
The job fairing-job-88xxk launched.
Waiting for fairing-job-88xxk-8ftpm to start...
Waiting for fairing-job-88xxk-8ftpm to start...
Waiting for fairing-job-88xxk-8ftpm to start...
Pod started running True
Cleaning up job fairing-job-88xxk...
```

Told K8s to pull the image and allocate resources. Ran the code, then cleaned it up

```
hello world!
```

The output of our function

In [2]:
import tensorflow as tf
from sklearn.datasets import make_regression
import numpy as np

In [3]:
class OLSModel(object):
    '''
    Use Tensorflow 1.14 to generate weights for an OLS regression via the formula
    
    w = (X'X)^-1 X'y
    
    ~y = w'X
    
    
    Loads random sample data via sklearn's make_regression
    
    '''
    
    def __init__(self):
        
        self.x,self.y = data,target = make_regression(n_samples=1000,n_features=1,bias=5,n_informative=1,random_state=1)
        self.w = None
        self.model = False
        
        
        
    def train(self, x=None, y=None):
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        if x==None or y==None:
            x,y = self.x,self.y
        
        ones = np.ones((x.shape[0],1))
        
        x = np.append(ones,x,axis=1)

        XtX = tf.matmul(x,x,transpose_a=True)
        XtY = tf.matmul(x,y.reshape(1000,1),transpose_a=True)

        w = tf.matmul(tf.matrix_inverse(XtX),XtY)
        self.w = np.array(sess.run(w))
        self.model=True
        
    def predict(self,x):
        if self.model==False:
            self.train()
            self.model=True
            
        sess = tf.Session()
        x = np.array(x)
        x = x.reshape((-1,1))
        x = np.append(np.ones((x.shape[0],1)),x,axis=1)
    
        return sess.run(tf.tensordot(x,self.w,1))
    

In [5]:
m = OLSModel()
m.train()

In [6]:
m.w

array([[ 5.        ],
       [38.69892343]])

In [12]:
base_image_tf = 'gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0'

In [13]:
fairing.config.set_builder('append',base_image=base_image_tf, registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

In [18]:
from kubeflow.fairing import TrainJob
train_job = TrainJob(OLSModel,
                     docker_registry=DOCKER_REGISTRY,base_docker_image=base_image_tf)
train_job.submit()

Can't determine namespace automatically. Using 'default' namespace but recomend to provide namespace explicitly. Using 'default' namespace might result in unable to mount some required secrets in cloud backends.
Using builder: <class 'kubeflow.fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_jcf0mist
/home/john/anaconda3/envs/kubeflow/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0'
Image successfully built in 0.4824504509961116s.
Pushing image localhost:5000/fairing-job:1A4C19E2...
Loading Docker credentials for repository 'localhost:5000/fairing-job:1A4C19E2'
Uploading localhost:5000/fairing-job:1A4C19E2
Layer sha256:e0a87f165c177d68c5dbcd5da0d1ab08bcaec973d956d92cef5417da7cf00a19 pushed.
Layer sha256:a31c3b1caa

BrokenPipeError: [Errno 32] Broken pipe

In [29]:
pred = fairing.PredictionEndpoint(trymodel,docker_registry=DOCKER_REGISTRY,base_docker_image=base_image_tf)

Can't determine namespace automatically. Using 'default' namespace but recomend to provide namespace explicitly. Using 'default' namespace might result in unable to mount some required secrets in cloud backends.
Using builder: <class 'kubeflow.fairing.builders.append.append.AppendBuilder'>


In [30]:
pred.create()

Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_u9a4dqgw
/home/john/anaconda3/envs/kubeflow/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0'
Image successfully built in 0.8494954710004095s.
Pushing image localhost:5000/fairing-job:52B0E688...
Loading Docker credentials for repository 'localhost:5000/fairing-job:52B0E688'


RemoteDisconnected: Remote end closed connection without response

# Additional Fairing Features

In [7]:
# notebook with Fire

# Caveats to Fairing

* Still in development; changes do happen
* Documentation is doesn't exist, only happens through examples which are not always up to date
* Favors the major cloud providers (GCP, Azure, AWS), with extra weight to GCP